In [1]:
import supervisely as sly
import os
import pandas as pd
from machine_learning.tasks.download_dataset_from_supervisely import download_datasets_from_supervisely

# Initialize Supervisely API
api_token = os.getenv("SUPERVISELY_API_TOKEN")
server_address = "https://glacier.enterprise.supervise.ly"
api = sly.Api(server_address, api_token)

# Define datasets to download
datasets_list = ["SCN-005-1.0", "SCN-005-1.1.a"]  # Example of small selection

# Download datasets from Supervisely
train_df = download_datasets_from_supervisely(api, "megamodel_labeling", datasets_list, parallel=True)


/home/barbara/miniconda3/envs/train/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 2536 annotations from dataset SCN-005-1.0 that are either objects or backgrounds.
Found 4308 annotations from dataset SCN-005-1.1.a that are either objects or backgrounds.


In [3]:
import numpy as np
import random
import sys
import os
sys.path.append('/home/barbara/glacier/layla/machine_learning')
from image_ops import read_image, grab_crop_from_image, write_image_to_local_disk

# Limit to the first 5000 random bounding boxes
# Exclude rows with class "BACKGROUND"
train_df = train_df[train_df["class"] != "BACKGROUND"].reset_index(drop=True)
train_df = train_df.sample(5000)

output_dir = "cropped_images"
os.makedirs(output_dir, exist_ok=True)

# Crop bounding boxes and save locally
for idx, row in train_df.iterrows():
    image_path = row["s3_uri"]  # Assumes `s3_uri` contains the path to the image
    class_name = row["class"]
    ymin, xmin, ymax, xmax = row["ymin"], row["xmin"], row["ymax"], row["xmax"]

    # Read image from S3
    image = read_image(image_path)

    # Normalize bounding box coordinates and crop
    image_height, image_width = image.shape[:2]
    box = (ymin / image_height, xmin / image_width, ymax / image_height, xmax / image_width)
    cropped_image = grab_crop_from_image(image, box)

    # Save cropped image locally
    cropped_image_path = os.path.join(output_dir, f"{class_name}_{idx}.jpg")
    write_image_to_local_disk(cropped_image, cropped_image_path)


KeyboardInterrupt: 